In [1]:
import ruamel.yaml as yl
import pandas as pd
import numpy as np

In [2]:
#in_file is the address for data model
in_file = "..."
yaml = yl.YAML(typ='safe')
with open(in_file, encoding='utf-8-sig') as file:
    data_model = yaml.load(file)

In [3]:
data_model

{'Nodes': {'institution': {'Props': ['institution_id', 'INSTITUTION_NAME']},
  'study_subject': {'Props': ['study_subject_id', 'SUBJECT_ID']},
  'demographic_data': {'Props': ['demographic_data_id',
    'RACE',
    'AGE_RANGE',
    'GENDER']},
  'prior_therapeutic_procedure': {'Props': ['prior_therapeutic_procedure_id',
    'PRIOR_THERAPY_RADIATION_TYPE',
    'PRIOR_THERAPY_CHEMO_AGENT_NAME',
    'PRIOR_THERAPY_PROCEDURE',
    'PRIOR_THERAPY_SURGERY_TUMOR_HISTOLOGY',
    'PRIOR_THERAPY_SURGERY_OUTCOME',
    'PRIOR_THERAPY_SURGERY_OUTCOME_2',
    'PT_NAME2',
    'PRIOR_THERAPY_SURGERY_OUTCOME_1',
    'PT_NAME1']},
  'on_therapeutic_procedure': {'Props': ['on_therapeutic_procedure_id',
    'ON_THERAPY_RADIATION_SITE',
    'ON_THERAPY_RADIATION_TYPE',
    'ON_THERAPY_CHEMO_AGENT_NAME',
    'ON_THERAPY_SURGERY_TITLE',
    'ON_THERAPY_SURGERY_HIST_DIAG',
    'ON_THERAPY_SURGERY_OUTCOME']},
  'diagnosis': {'Props': ['diagnosis_id', 'DISEASE_TYPE', 'WHO_GRADE']},
  'outcome': {'Props': ['outc

In [4]:
#read in the data
data = pd.read_csv(..., sep='\t')

In [5]:
data.columns

Index(['SUBJECT_ID', 'PARENT_ID', 'TYPE', 'TIMEPOINT', 'AGE_RANGE', 'GENDER',
       'EVENT_OS', 'OVERALL_SURVIVAL_MONTHS', 'DISEASE_TYPE', 'WHO_GRADE',
       'RACE', 'INSTITUTION_NAME', 'KARNOFSKY', 'NEURO_EXAM_SCORE',
       'DISEASE_EVAL_MRI', 'STEROID_DOSE_STATUS', 'ANTI_CONVULSANT_STATUS',
       'PRIOR_THERAPY_RADIATION_TYPE', 'PRIOR_THERAPY_CHEMO_AGENT_NAME',
       'PRIOR_THERAPY_SURGERY_TUMOR_HISTOLOGY', 'PRIOR_THERAPY_PROCEDURE',
       'PRIOR_THERAPY_SURGERY_OUTCOME', 'ON_THERAPY_RADIATION_SITE',
       'ON_THERAPY_RADIATION_TYPE', 'ON_THERAPY_CHEMO_AGENT_NAME',
       'ON_THERAPY_SURGERY_HIST_DIAG', 'ON_THERAPY_SURGERY_TITLE',
       'ON_THERAPY_SURGERY_OUTCOME', 'PRIOR_THERAPY_SURGERY_OUTCOME_2',
       'PT_NAME2', 'PRIOR_THERAPY_SURGERY_OUTCOME_1', 'PT_NAME1', 'GENE_EXP',
       'COPY_NUMBER'],
      dtype='object')

In [6]:
table = data.copy()
for i in data_model["Nodes"].keys():
    colnames = list(table.columns.intersection(data_model["Nodes"][i]['Props']))
    if colnames == []:
        print("Nodes \'" + i + "\' is not in table")
        pass
    else:    
        IDcol = i + '_id'
        table.insert(0, IDcol, table.groupby(colnames, dropna= False).ngroup())
        table[IDcol] =  IDcol + '_' + table[IDcol].astype(str)    
print(table.head(3))

Nodes 'file' is not in table
       clinical_evaluation_id      outcome_id     diagnosis_id  \
0  clinical_evaluation_id_192  outcome_id_373  diagnosis_id_16   
1  clinical_evaluation_id_192  outcome_id_373  diagnosis_id_16   
2  clinical_evaluation_id_192  outcome_id_373  diagnosis_id_16   

       on_therapeutic_procedure_id      prior_therapeutic_procedure_id  \
0  on_therapeutic_procedure_id_172  prior_therapeutic_procedure_id_196   
1  on_therapeutic_procedure_id_172  prior_therapeutic_procedure_id_196   
2  on_therapeutic_procedure_id_172  prior_therapeutic_procedure_id_196   

      demographic_data_id     study_subject_id     institution_id SUBJECT_ID  \
0  demographic_data_id_85  study_subject_id_14  institution_id_11     860772   
1  demographic_data_id_85   study_subject_id_0  institution_id_11    3089254   
2  demographic_data_id_85   study_subject_id_1  institution_id_11    3495697   

   PARENT_ID  ... ON_THERAPY_CHEMO_AGENT_NAME  ON_THERAPY_SURGERY_HIST_DIAG  \
0        

In [7]:
# Search Src and Dst to determine if a parent ID column need to be add
for i in data_model["Nodes"].keys():
    colnames = list(table.columns.intersection(data_model["Nodes"][i]['Props']))
    if colnames == []:
        print("Nodes \'" + i + "\' is not in table")
        pass
    else:
        print("Nodes \'" + i + "\' table:")
        subtable = table.loc[:,colnames]
        for x in data_model["Relationships"].keys():
            for a in data_model["Relationships"][x]['Ends']:
                b = a.get('Src')
                if b == i:
                    parentname = a.get('Dst')
                    ParID = parentname + '_id'
                    NewParID = parentname + '.' + parentname + '_id'
                    subtable.insert(0, NewParID, table[ParID])               
        subtable.insert(0, 'type', i)
        print(subtable.head(3))
        filename = i + '_file.txt'
        subtable.to_csv('./output/' + filename, sep = '\t', index = False)

Nodes 'institution' table:
          type study_subject.study_subject_id     institution_id  \
0  institution            study_subject_id_14  institution_id_11   
1  institution             study_subject_id_0  institution_id_11   
2  institution             study_subject_id_1  institution_id_11   

  INSTITUTION_NAME  
0              TJU  
1              TJU  
2              TJU  
Nodes 'study_subject' table:
            type     study_subject_id SUBJECT_ID
0  study_subject  study_subject_id_14     860772
1  study_subject   study_subject_id_0    3089254
2  study_subject   study_subject_id_1    3495697
Nodes 'demographic_data' table:
               type study_subject.study_subject_id     demographic_data_id  \
0  demographic_data            study_subject_id_14  demographic_data_id_85   
1  demographic_data             study_subject_id_0  demographic_data_id_85   
2  demographic_data             study_subject_id_1  demographic_data_id_85   

  AGE_RANGE GENDER RACE  
0       NaN         